In [1]:
#Update config.ymal
import os

In [2]:
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification'

In [6]:
import os
from dataclasses import dataclass
from pathlib import Path
import yaml

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



Creating directory: artifacts
Creating directory: artifacts\data_ingestion
File already exists of size: 26053046 bytes
Extracted to artifacts\data_ingestion


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
def read_yaml(path_to_yaml: Path) -> dict:
    with open(path_to_yaml) as yaml_file:
        content = yaml.safe_load(yaml_file)
    return content

def create_directories(paths: list):
    for path in paths:
        print(f"Creating directory: {path}")  # Debugging output
        os.makedirs(path, exist_ok=True)

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config['artifacts_root'])])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        create_directories([Path(config['root_dir'])])
        return DataIngestionConfig(
            root_dir=Path(config['root_dir']),
            source_URL=config['source_URL'],
            local_data_file=Path(config['local_data_file']),
            unzip_dir=Path(config['unzip_dir'])
        )

import urllib.request as request
import zipfile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not self.config.local_data_file.exists():
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded with following info: \n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)} bytes")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        print(f"Extracted to {unzip_path}")

# Pipeline creation
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print(f"An error occurred: {e}")


[2024-05-19 22:07:07,296: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-19 22:07:07,298: INFO: common: yaml file: params.yaml loaded successfully]


TypeError: expected str, bytes or os.PathLike object, not type